In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [2]:
pf1 = pd.read_excel('C:/Users/omesh/Desktop/CAPSTONE/Estimize.xlsx')

In [3]:
pf2 = pd.read_excel('C:/Users/omesh/Downloads/moreEstimizeReleases.xlsx')

In [4]:
earning_spz = ((pf1['ACT EPS'] - pf1['ESZ EPS Est']) / abs(pf1['ESZ EPS Est'])) * 100

In [5]:
earning_spz.shape

(2158,)

In [6]:
earning_spz = pd.Series(earning_spz, name = 'Earnings Surprise %').to_frame()

In [7]:
pf1 = pd.concat([pf1, earning_spz], axis = 1, join = 'outer')

In [8]:
pf1 = pf1.drop(pf1[pf1.shortName == 'setup'].index)

In [15]:
pf1 = pf1.drop(pf1[pf1.ID == '4f066ccec6efa653ac00ec8c'].index)

In [9]:
pf1['Release Date'] = pd.to_datetime(pf1['Release Date'])

In [10]:
pf1.shape

(2150, 24)

In [11]:
pf1 = pf1[pf1['Earnings Surprise %'].notnull()]

In [13]:
print(pf1.dtypes)
pf2.dtypes

Fiscal Year                   float64
Fiscal Quarter                float64
ACT EPS                       float64
WS EPS Est                    float64
ESZ EPS Est                   float64
Release Date           datetime64[ns]
ID                             object
symbol                         object
fiscalDate                     object
Actual EPS vs WS               object
WS EPS Dif                    float64
WS EPS % Dif                  float64
Actual EPS vs ESZ              object
ESZ EPS Dif                   float64
ESZ EPS % Dif                 float64
close                         float64
Per1d                         float64
Per2d                         float64
Per3d                         float64
Per5d                         float64
shortName                      object
refSectorShort                 object
mktCap                         object
Earnings Surprise %           float64
dtype: object


Fiscal Year                      int64
Fiscal Quarter                   int64
ACT EPS                        float64
ACT Revenue                    float64
ESZ EPS Est UW                 float64
ESZ REV Est UW                 float64
WS REV Est                     float64
WS EPS Est                     float64
ESZ REV Est                    float64
ESZ EPS Est                    float64
Release Date                    object
ID                               int64
Symbol                          object
Actual +/- WS                  float64
Actual - WS EPS EST            float64
rankNextRelease                float64
nextRelease                    float64
listName                        object
rankLastRelease                float64
nextWsEpsEst                   float64
nextEszEpsEst                  float64
nextWsRevEst                   float64
nextEszRevEst                  float64
lastActEPS                     float64
lastActRev                     float64
countReleases            

In [16]:
pf1['ID'] = pf1['ID'].astype(int)

In [17]:
pf0 = pd.merge(pf1, pf2[["ID","Release Date"]], on="ID", how="left")

In [18]:
pf0.head()

,Fiscal Year,Fiscal Quarter,ACT EPS,WS EPS Est,ESZ EPS Est,Release Date_x,ID,symbol,fiscalDate,Actual EPS vs WS,...,close,Per1d,Per2d,Per3d,Per5d,shortName,refSectorShort,mktCap,Earnings Surprise %,Release Date_y
0,2018.0,1.0,3.89,3.843,3.844296,2018-02-01,131754,AAPL,2018 Q1,Beat,...,167.78,-4.339015,-6.729050,-2.831088,-7.527715,SPX 100,E Technology,Mega Cap,1.188886,2018-02-01T16:00:00-05:00
1,2018.0,2.0,2.73,2.670,2.724264,2018-05-01,133987,AAPL,2018 Q2,Beat,...,169.10,4.417504,4.606742,8.710822,10.023655,SPX 100,E Technology,Mega Cap,0.210568,2018-05-01T16:00:00-04:00
2,2018.0,3.0,2.34,2.173,2.217906,2018-07-31,136224,AAPL,2018 Q3,Beat,...,190.29,5.891008,8.986284,9.301592,8.839140,SPX 100,E Technology,Mega Cap,5.504926,2018-07-31T16:00:00-04:00
3,2018.0,4.0,2.91,2.788,2.842092,2018-11-01,146839,AAPL,2018 Q4,Beat,...,222.22,-6.633066,-9.283593,-8.302583,-6.178562,SPX 100,E Technology,Mega Cap,2.389383,2018-11-01T16:00:00-04:00
4,2019.0,1.0,4.18,4.170,4.183036,2019-01-29,149518,AAPL,2019 Q1,Beat,...,154.68,6.833463,7.602793,7.654513,12.606672,SPX 100,E Technology,Mega Cap,-0.072571,2019-01-29T16:00:00-05:00


In [19]:
pf0.shape

(1306, 25)

In [24]:
pf = pf0.drop(['Release Date_y'], axis = 1, inplace = False)

In [25]:
pf.isnull().sum()

Fiscal Year             0
Fiscal Quarter          0
ACT EPS                 0
WS EPS Est              2
ESZ EPS Est             0
Release Date_x          0
ID                      0
symbol                  0
fiscalDate              0
Actual EPS vs WS        2
WS EPS Dif              2
WS EPS % Dif            7
Actual EPS vs ESZ       0
ESZ EPS Dif             0
ESZ EPS % Dif           5
close                  65
Per1d                  65
Per2d                  65
Per3d                  65
Per5d                  65
shortName              65
refSectorShort          0
mktCap                  0
Earnings Surprise %     0
dtype: int64

In [28]:
pf.shape

(1235, 24)

In [27]:
pf = pf.dropna()

In [29]:
pf['m_cluster'] = np.zeros(np.shape(pf)[0])
pf['dir_cluster'] = np.zeros(np.shape(pf)[0])

In [30]:
for i in range(np.shape(pf)[0]):
    if pf['Per5d'].iloc[i]>=-10 and pf['Per5d'].iloc[i] <0:
        pf['m_cluster'].iloc[i] = 1
    elif pf['Per5d'].iloc[i] <= 10 and pf['Per5d'].iloc[i] >=0:
        pf['m_cluster'].iloc[i] = 2
    elif pf['Per5d'].iloc[i] > 10:
        pf['m_cluster'].iloc[i] = 3

C:\Users\omesh\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
pf['m_cluster']

0       1.0
1       3.0
2       2.0
3       1.0
4       3.0
       ... 
1236    0.0
1237    0.0
1238    0.0
1239    2.0
1240    1.0
Name: m_cluster, Length: 1235, dtype: float64

In [33]:
for i in range(np.shape(pf)[0]):
    diff = pf['Per5d'].iloc[i] - pf['Per1d'].iloc[i]
    if diff>2:
        pf['dir_cluster'].iloc[i] = 2
    elif diff<-2:
        pf['dir_cluster'].iloc[i] = 0
    else:
        pf['dir_cluster'].iloc[i] = 1


In [34]:
pf['dir_cluster']

0       0.0
1       2.0
2       2.0
3       1.0
4       2.0
       ... 
1236    0.0
1237    0.0
1238    2.0
1239    1.0
1240    0.0
Name: dir_cluster, Length: 1235, dtype: float64

In [35]:
pf['cluster_pair'] = '(' + pf["dir_cluster"].astype(str) +','+ (pf["m_cluster"].astype(int)).astype(str) + ')'

In [37]:
pf['cluster_pair'] = '(' + (pf["dir_cluster"].astype(int)).astype(str) +','+ (pf["m_cluster"].astype(int)).astype(str) + ')'

In [38]:
pf[['Per5d','dir_cluster','m_cluster','cluster_pair']]

,Per5d,dir_cluster,m_cluster,cluster_pair
0,-7.527715,0.0,1.0,"(0,1)"
1,10.023655,2.0,3.0,"(2,3)"
2,8.839140,2.0,2.0,"(2,2)"
3,-6.178562,1.0,1.0,"(1,1)"
4,12.606672,2.0,3.0,"(2,3)"
...,...,...,...,...
1236,-11.770428,0.0,0.0,"(0,0)"
1237,-19.810933,0.0,0.0,"(0,0)"
1238,-26.783920,2.0,0.0,"(2,0)"
1239,7.178751,1.0,2.0,"(1,2)"


In [39]:
pf['close_price_variation'] = np.zeros(np.shape(pf)[0])

In [40]:
for i in range(np.shape(pf)[0]):
    '''DIRECTION'''
    
    if pf['dir_cluster'].iloc[i] == 2:
        last = 'Increasing'
    elif pf['dir_cluster'].iloc[i] == 1:
        last = 'Stable'
    else:
        last = 'Decreasing'
        
    '''MAGNITUDE'''
    
    if pf['m_cluster'].iloc[i] == 3:
        first = 'Highly Positive'
    elif pf['m_cluster'].iloc[i] == 2:
        first = 'Positive'
    elif pf['m_cluster'].iloc[i] == 1:
        first = 'Negative'
    else:
        first = 'Highly Negative'
        
    pf['close_price_variation'].iloc[i] = first + ' & ' + last

In [41]:
pf[['cluster_pair','close_price_variation']]

,cluster_pair,close_price_variation
0,"(0,1)",Negative & Decreasing
1,"(2,3)",Highly Positive & Increasing
2,"(2,2)",Positive & Increasing
3,"(1,1)",Negative & Stable
4,"(2,3)",Highly Positive & Increasing
...,...,...
1236,"(0,0)",Highly Negative & Decreasing
1237,"(0,0)",Highly Negative & Decreasing
1238,"(2,0)",Highly Negative & Increasing
1239,"(1,2)",Positive & Stable


In [42]:
pf.head()

,Fiscal Year,Fiscal Quarter,ACT EPS,WS EPS Est,ESZ EPS Est,Release Date_x,ID,symbol,fiscalDate,Actual EPS vs WS,...,Per3d,Per5d,shortName,refSectorShort,mktCap,Earnings Surprise %,m_cluster,dir_cluster,cluster_pair,close_price_variation
0,2018.0,1.0,3.89,3.843,3.844296,2018-02-01,131754,AAPL,2018 Q1,Beat,...,-2.831088,-7.527715,SPX 100,E Technology,Mega Cap,1.188886,1.0,0.0,"(0,1)",Negative & Decreasing
1,2018.0,2.0,2.73,2.670,2.724264,2018-05-01,133987,AAPL,2018 Q2,Beat,...,8.710822,10.023655,SPX 100,E Technology,Mega Cap,0.210568,3.0,2.0,"(2,3)",Highly Positive & Increasing
2,2018.0,3.0,2.34,2.173,2.217906,2018-07-31,136224,AAPL,2018 Q3,Beat,...,9.301592,8.839140,SPX 100,E Technology,Mega Cap,5.504926,2.0,2.0,"(2,2)",Positive & Increasing
3,2018.0,4.0,2.91,2.788,2.842092,2018-11-01,146839,AAPL,2018 Q4,Beat,...,-8.302583,-6.178562,SPX 100,E Technology,Mega Cap,2.389383,1.0,1.0,"(1,1)",Negative & Stable
4,2019.0,1.0,4.18,4.170,4.183036,2019-01-29,149518,AAPL,2019 Q1,Beat,...,7.654513,12.606672,SPX 100,E Technology,Mega Cap,-0.072571,3.0,2.0,"(2,3)",Highly Positive & Increasing


In [43]:
pf.to_csv('PF_Clustering1.csv', index = False, encoding='utf-8-sig')

In [44]:
cl_avg = pf.groupby('close_price_variation', as_index = False).agg({'Per1d': 'mean','Per2d': 'mean','Per3d': 'mean','Per5d': 'mean'})

In [45]:
cl_avg

,close_price_variation,Per1d,Per2d,Per3d,Per5d
0,Highly Negative & Decreasing,-8.816956,-11.758343,-13.564116,-15.552688
1,Highly Negative & Increasing,-25.268218,-23.272759,-22.856126,-20.867139
2,Highly Negative & Stable,-16.520170,-14.738777,-15.911419,-16.442692
3,Highly Positive & Decreasing,22.510622,20.234864,19.913810,16.736065
4,Highly Positive & Increasing,9.789180,12.668195,14.379042,17.449232
5,Highly Positive & Stable,16.118315,18.244286,17.842399,16.662897
6,Negative & Decreasing,-0.403918,-1.541601,-2.637052,-4.736635
7,Negative & Increasing,-6.874240,-5.366106,-4.248479,-2.873414
8,Negative & Stable,-2.589826,-2.609996,-2.576960,-2.868564
9,Positive & Decreasing,7.708479,6.562870,5.519745,3.741496


In [46]:
cl_avg.to_csv('PF_Cluster1.csv', index = False, encoding='utf-8-sig')

In [47]:
pf['symbol'].nunique()

168